Importing the Dependencies

In [16]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import joblib

In [17]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Asanka\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
# printing the stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Pre-processing

In [19]:
# loading the dataset to a pandas DataFrame
news_dataset = pd.read_csv('dataset/train.csv')

In [20]:
news_dataset.shape

(20800, 5)

In [21]:
# print the first 5 rows of the dataframe
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [22]:
# counting the number of missing values in the dataset
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [23]:
# replacing the null values with empty string
news_dataset = news_dataset.fillna('')

In [24]:
# merging the author name and news title
news_dataset['content'] = news_dataset['author']+' '+news_dataset['title']

In [27]:
news_dataset.head(5)

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [28]:
news_dataset.isnull().sum()

id         0
title      0
author     0
text       0
label      0
content    0
dtype: int64

In [17]:
print(news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
5        Daniel Nussbaum Jackie Mason: Hollywood Would ...
6         Life: Life Of Luxury: Elton John’s 6 Favorite...
7        Alissa J. Rubin Benoît Hamon Wins French Socia...
8         Excerpts From a Draft Script for Donald Trump...
9        Megan Twohey and Scott Shane A Back-Channel Pl...
10       Aaron Klein Obama’s Organizing for Action Part...
11       Chris Tomlinson BBC Comedy Sketch "Real Housew...
12       Amando Flavio Russian Researchers Discover Sec...
13       Jason Ditz US Officials See No Link Between Tr...
14       AnotherAnnie Re: Yes, There Are Paid Governmen...
15       Jack Williams In Major League Soccer, Argentin...
16       Michael Corkery and Stacy Cowley Wells Fargo C.

In [29]:
news_dataset.head(5)

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [31]:
# separating the data & label
X = news_dataset.drop(columns='label', axis=1)
Y = news_dataset['label']

In [32]:
X.head(5)

,id,title,author,text,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,Howard Portnoy Iranian woman jailed for fictio...


In [19]:
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
5          5  Jackie Mason: Hollywood Would Love Trump if He...   
6          6  Life: Life Of Luxury: Elton John’s 6 Favorite ...   
7          7  Benoît Hamon Wins French Socialist Party’s Pre...   
8          8  Excerpts From a Draft Script for Donald Trump’...   
9          9  A Back-Channel Plan for Ukraine and Russia, Co...   
10        10  Obama’s Organizing for Action Partners with So...   
11        11  BBC Comedy Sketch "Real Housewives of ISIS" Ca...   
12        12  Russian Researchers Discover Secret Nazi Milit...   
13        13  US Officials See No Link Between Trump and Russi

Stemming:

Stemming is the process of reducing a word to its Root word

example:
actor, actress, acting --> act

In [20]:
port_stem = PorterStemmer()

In [21]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [22]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [23]:
print(news_dataset['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
5        daniel nussbaum jacki mason hollywood would lo...
6        life life luxuri elton john favorit shark pict...
7        alissa j rubin beno hamon win french socialist...
8        excerpt draft script donald trump q ampa black...
9        megan twohey scott shane back channel plan ukr...
10       aaron klein obama organ action partner soro li...
11       chri tomlinson bbc comedi sketch real housew i...
12       amando flavio russian research discov secret n...
13              jason ditz us offici see link trump russia
14       anotheranni ye paid govern troll social media ...
15       jack william major leagu soccer argentin find ...
16       michael corkeri staci cowley well fargo chief .

In [34]:
news_dataset.head(5)

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [33]:
#separating the data and label
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [25]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [26]:

# data_ = np.asarray(X)
# data_
# data_.tofile('data.csv',format='%.18e')

In [27]:
print(Y)

[1 0 1 ... 0 1 1]


In [28]:
Y.shape

(20800,)

In [30]:
X.shape

(20800,)

In [37]:
X

array(['darrel lucu hous dem aid even see comey letter jason chaffetz tweet',
       'daniel j flynn flynn hillari clinton big woman campu breitbart',
       'consortiumnew com truth might get fire', ...,
       'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time',
       'alex ansari nato russia hold parallel exercis balkan',
       'david swanson keep f aliv'], dtype=object)

In [31]:
type(X)

numpy.ndarray

In [66]:
from numpy import save
# define data
data = X
# save to npy file
save('model/data.npy', data)

In [67]:
# load numpy array from npy file
# from numpy import load
# # load array
# X1 = load('model/data.npy',allow_pickle=True)
# # print the array


In [64]:
# X1

array(['darrel lucu hous dem aid even see comey letter jason chaffetz tweet',
       'daniel j flynn flynn hillari clinton big woman campu breitbart',
       'consortiumnew com truth might get fire', ...,
       'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time',
       'alex ansari nato russia hold parallel exercis balkan',
       'david swanson keep f aliv'], dtype=object)

In [34]:

# pd.DataFrame(X).to_csv("model/x.csv")

In [44]:
# from numpy import genfromtxt
# # my_data = genfromtxt('model/x.csv', delimiter=',')

# my_data = pd.read_csv('model/x.csv')

In [52]:
# my_data.head(5)

,Unnamed: 0,0
0,0,darrel lucu hous dem aid even see comey letter...
1,1,daniel j flynn flynn hillari clinton big woman...
2,2,consortiumnew com truth might get fire
3,3,jessica purkiss civilian kill singl us airstri...
4,4,howard portnoy iranian woman jail fiction unpu...


In [53]:
# my_data.head(5)
# my_data1 = my_data['0']

In [54]:
# my_data

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
5        daniel nussbaum jacki mason hollywood would lo...
6        life life luxuri elton john favorit shark pict...
7        alissa j rubin beno hamon win french socialist...
8        excerpt draft script donald trump q ampa black...
9        megan twohey scott shane back channel plan ukr...
10       aaron klein obama organ action partner soro li...
11       chri tomlinson bbc comedi sketch real housew i...
12       amando flavio russian research discov secret n...
13              jason ditz us offici see link trump russia
14       anotheranni ye paid govern troll social media ...
15       jack william major leagu soccer argentin find ...
16       michael corkeri staci cowley well fargo chief .

NameError: name 'fdgdfgdfgd' is not defined

In [ ]:
X

In [35]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)


In [37]:
print(X)

  (0, 23355)	0.18006497451107856
  (0, 22649)	0.26575278886038384
  (0, 22289)	0.3484071341454308
  (0, 19171)	0.22537992364975484
  (0, 12902)	0.3024224900242886
  (0, 12528)	0.24883399099107747
  (0, 11409)	0.20615188166061463
  (0, 11307)	0.1532265401605094
  (0, 10387)	0.1844880289323935
  (0, 7574)	0.23047267305353566
  (0, 6145)	0.24677171892553343
  (0, 5800)	0.2502787762405247
  (0, 5508)	0.2993429551929777
  (0, 4507)	0.20531415441295317
  (0, 3862)	0.30579573877221844
  (0, 904)	0.26354004814013343
  (1, 23748)	0.2966210296019264
  (1, 15149)	0.15862263711495958
  (1, 10134)	0.18787145765749733
  (1, 8420)	0.7045992054867243
  (1, 5469)	0.2624012615566619
  (1, 4298)	0.19024289659874757
  (1, 3509)	0.37751839443307017
  (1, 3075)	0.15310531118537438
  (1, 2544)	0.2899843833664323
  :	:
  (20797, 11262)	0.12516633187998083
  (20797, 10427)	0.20792477683235197
  (20797, 5559)	0.20389975589596085
  (20797, 3394)	0.14456424605079038
  (20797, 2273)	0.31989436828531154
  (20797, 1

In [ ]:
print(X)

Splitting the dataset to training & test data

In [38]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

Training the Model: Logistic Regression

In [39]:
model = LogisticRegression()

In [40]:
print(X_train)

  (0, 991)	0.3108936537038392
  (0, 4268)	0.35257385602238933
  (0, 4886)	0.3218149918353991
  (0, 14544)	0.28683044607659075
  (0, 14657)	0.08939851479196703
  (0, 15149)	0.13521763024214778
  (0, 16959)	0.281126113562025
  (0, 17807)	0.43134727637953557
  (0, 18583)	0.2777120045701306
  (0, 19791)	0.32505384074761434
  (0, 21564)	0.07370617002885245
  (0, 21736)	0.09168254405563406
  (0, 23915)	0.3108936537038392
  (0, 23989)	0.09249239245759266
  (1, 12)	0.1949928443128574
  (1, 1203)	0.14999077977392733
  (1, 1233)	0.08205629736174584
  (1, 1803)	0.30747202041406685
  (1, 1850)	0.2591914353791728
  (1, 3485)	0.30747202041406685
  (1, 4218)	0.30747202041406685
  (1, 6506)	0.18952348674492747
  (1, 8123)	0.2591914353791728
  (1, 10397)	0.1404878194768246
  (1, 14085)	0.30747202041406685
  :	:
  (16637, 22152)	0.380661170099508
  (16637, 23989)	0.08162393566901983
  (16638, 958)	0.23679976988599757
  (16638, 2322)	0.24133054039442228
  (16638, 5786)	0.5616120123127117
  (16638, 9170)	

In [41]:
print(Y_train)

[0 0 0 ... 0 0 1]


In [42]:
print(X_train.shape,"   ",Y_train.shape)

(16640, 25173)     (16640,)


In [43]:
model.fit(X_train, Y_train)

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
joblib.dump(model,'model/fake_news_model.sav')

Evaluation

accuracy score

In [44]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [45]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.9871995192307692


In [46]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [47]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.9790865384615385


Making a Predictive System

In [50]:
X_new = X_test[5]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[1]
The news is Fake


In [51]:
print(Y_test[3])

0


In [52]:
print(X_test[3])

  (0, 3075)	0.0999341588255635
  (0, 3394)	0.1290701398684749
  (0, 3757)	0.2353995578665707
  (0, 3901)	0.19916118717179757
  (0, 4285)	0.21101063742479315
  (0, 5786)	0.4798912624808988
  (0, 8463)	0.30075398340678067
  (0, 10202)	0.2560824601482711
  (0, 11383)	0.17482331013137414
  (0, 15860)	0.3079444925959449
  (0, 15896)	0.3172146712470392
  (0, 16857)	0.2675758438774059
  (0, 18181)	0.3172146712470392
  (0, 20780)	0.2217735487185802


In [53]:
news1 = ['Specter of Trump Loosens Tongues, if Not Purse Strings, in Silicon Valley - The New York Times'
            ,'David Streitfeld'
            ,'adasd']

In [54]:
def model_result(news):
  
        import joblib
        from sklearn.feature_extraction.text import TfidfVectorizer
        from numpy import load
        # load array
        X1 = load('model/data.npy',allow_pickle=True)
        model_l = joblib.load('model/fake_news_model.sav')

        vectorizer = TfidfVectorizer()
        vectorizer.fit(X1)
        X_new = vectorizer.transform(news)

        prediction = model_l.predict(X_new)
        #print(prediction)

        if (prediction[0]==0):
          # print('The news is Real')
          return True
        else:
          # print('The news is Fake')
          return False